In [1]:
#iniciar spark
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

conf = SparkConf()
#conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.2.2')
conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.2.2,com.microsoft.azure:spark-mssql-connector_2.12:1.2.0')
conf.set('spark.hadoop.fs.s3a.aws.credentials.provider', 'com.amazonaws.auth.InstanceProfileCredentialsProvider')
spark = SparkSession.builder.config(conf=conf).getOrCreate()

:: loading settings :: url = jar:file:/usr/local/lib/python3.7/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
com.microsoft.azure#spark-mssql-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-53e1e17c-376a-4c05-a5dd-246f28b7842c;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.2.2 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.563 in central
	found com.microsoft.azure#spark-mssql-connector_2.12;1.2.0 in central
	found com.microsoft.sqlserver#mssql-jdbc;8.4.1.jre8 in central
:: resolution report :: resolve 406ms :: artifacts dl 20ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.563 from central in [default]
	com.microsoft.azure#spark-mssql-connector_2.12;1.2.0 from central in [default]
	com.microsoft.sqlserver#mssql-jdbc;8.4.1.jre8 from central in [default]
	org.apache.hadoop#hadoop-aws;3.2.2 from central in [default]
	-------------------------------------------

In [5]:
import os

S3_address_gross = os.environ['S3ADRESS_GROSS']
S3_address_treated = os.environ['S3ADRESS_TREATED']
S3_address_client = os.environ['S3ADRESS_CLIENT']
password_sql = os.environ['PASSWORD_SQL']
user_sql = os.environ['USER_SQL']

In [1]:
df_unidade_federativa = spark.read.option("multiline","true")\
                        .json(f"{S3_address_gross}/estados_soja_1974_2020.json")
df_regiao = spark.read.option('delimiter', ',')\
    .option('header', 'true').csv(f"{S3_address_gross}/estados.csv")

NameError: name 'spark' is not defined

In [2]:
df_unidade_federativa = df_unidade_federativa.na.drop()

NameError: name 'df_unidade_federativa' is not defined

In [8]:
df = df_unidade_federativa.join(df_regiao, (df_regiao.nome ==
                                       df_unidade_federativa['Unidade da Federação'])
                          )['Ano', 'Toneladas', 'Unidade da Federação','regiao'].show()

+----+---------+--------------------+------+
| Ano|Toneladas|Unidade da Federação|regiao|
+----+---------+--------------------+------+
|1974|        -|            Rondônia| Norte|
|1975|        -|            Rondônia| Norte|
|1976|        -|            Rondônia| Norte|
|1977|        -|            Rondônia| Norte|
|1978|        -|            Rondônia| Norte|
|1979|        -|            Rondônia| Norte|
|1980|        -|            Rondônia| Norte|
|1981|        -|            Rondônia| Norte|
|1982|        -|            Rondônia| Norte|
|1983|        -|            Rondônia| Norte|
|1984|        -|            Rondônia| Norte|
|1985|      677|            Rondônia| Norte|
|1986|     1160|            Rondônia| Norte|
|1987|     2050|            Rondônia| Norte|
|1988|     9666|            Rondônia| Norte|
|1989|    18310|            Rondônia| Norte|
|1990|     9252|            Rondônia| Norte|
|1991|     1932|            Rondônia| Norte|
|1992|     9852|            Rondônia| Norte|
|1993|    

In [4]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, FloatType

_df = df

_df = _df.select(
    col('Ano').cast(IntegerType()).alias('ANO'),
    col('Toneladas').cast(IntegerType()).alias('TONELADAS'),
    col('Unidade da Federação').alias('UF'),
    col('regiao').alias('REGIAO')
)

_df.write \
   .mode('overwrite') \
   .csv(f'{S3_address_treated}/brazil_soybean_regiao')

22/05/30 02:12:44 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/05/30 02:12:45 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/05/30 02:12:45 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.


In [5]:
dataframe = spark.read.csv(f"{S3_address_treated}/brazil_soybean_regiao")

In [6]:
dataframe = dataframe.withColumnRenamed("_c0", "Ano")
dataframe = dataframe.withColumnRenamed("_c1", "Toneladas")
dataframe = dataframe.withColumnRenamed("_c2", "UF")
dataframe = dataframe.withColumnRenamed("_c3", "Regiao")

In [7]:

dataframe = dataframe.select(
    col('Ano').cast(IntegerType()).alias('ANO'),
    col('Toneladas').cast(IntegerType()).alias('TONELADAS'),
    col('Regiao').alias('REGIAO'),
     col('UF').alias('UF')
).fillna(0)

In [8]:
dataframe[(dataframe['Regiao'] == 'Centro-Oeste') & (dataframe['Ano'] == '1974')]

,Ano,Toneladas,UF,Regiao
447,1974,0,Mato Grosso do Sul,Centro-Oeste
494,1974,307010,Mato Grosso,Centro-Oeste
541,1974,99000,Goiás,Centro-Oeste
588,1974,23,Distrito Federal,Centro-Oeste


In [9]:
grouped_by_regiao = dataframe.groupBy(['REGIAO','ANO']).sum('TONELADAS').withColumnRenamed("sum(TONELADAS)", "TONELADAS") 

In [13]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, FloatType

_df = spark.createDataFrame(sum_dataframe_groupby)

_df = _df.select(
    col('Ano').cast(IntegerType()).alias('ANO'),
    col('Toneladas').cast(IntegerType()).alias('TONELADAS'),
    col('Regiao').alias('REGIAO'),
    col('UF').alias("UF")
)

_df.write \
   .mode('overwrite') \
   .csv(f'{S3_address_client}/brazil_soybean_regiao')

_df.write \
 .format('com.microsoft.sqlserver.jdbc.spark') \
 .mode('overwrite') \
 .option('driver', 'com.microsoft.sqlserver.jdbc.SQLServerDriver') \
 .option('url', 'jdbc:sqlserver://poupay-01.database.windows.net;databaseName=poupay;') \
 .option('dbtable', 'UF_REGIAO_SOJA_IBGE') \
 .option('user', user_sql) \
 .option('password', password_sql) \
 .save()

22/05/30 02:19:00 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/05/30 02:19:00 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/05/30 02:19:00 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
